In [1]:
from netin.models import DPAHModel

In [2]:
#TODO: check icm diffusion model, error when generating the random values, always reusing the same ones over and over again!!!!
#TODO: reduce seed size, to have more variability in the results
#TODO: Change budget amount, and see if one then have more variability in the results

In [3]:
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set


from src.seed import (seed_degree, 
    seed_polarized, 
    seed_random, 
    seed_polarized_centrality_mixed, 
    seed_polarized_centrality, 
    seed_polarized_degree, 
    seed_centrality_mixed, 
    seed_centrality, 
    seed_influence_maximization
)


from evaluation import evaluate_all_seeds

This model connects a new node to the existing nodes biased by their group assignment. Homophily values of $h > 0.5$ indicate homophily, meaning that nodes prefer to connect to their own group. Heterophily is specified by $h < 0.5$. Nodes tend to connect to the other group.

Instead of the number of links per each new node, the directed graph models parameterize the density of the final network by the parameter $d$. Additionally, we need to define activity levels for outgoing links for both the minority and majority group as $plo_m/M$

In [4]:
N = 10_000 # Number of total nodes
f_m = 0.3 # Fraction of minority nodes
h = 0.1 # Homophilic
d = 0.001 # desity of the graph
plo = 2.0
#final number of edges will be: e = d * n (n-1)

In [5]:
# DPAH graphs:
# Homophilic h > 0.5
# Neutral h = 0.5
# Heterophilic h < 0.5

homophily_values = [0.1, 0.5, 0.9]

In [6]:
dpa_model = DPAHModel(N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h, seed=12345)
dpa_graph = dpa_model.simulate()

In [7]:
G = dpa_graph.to_nxgraph() # This creates a copy of the graph

for node, attr in G.nodes(data=True):
    attr['color'] = attr.pop('minority')

for node in G.nodes():
        in_degree = G.in_degree(node)
        for neighbor in G.successors(node):
            G[node][neighbor]["weight"] = 1 / in_degree if in_degree > 0 else 1

In [8]:
seed_size = G.number_of_nodes() // 40
print(f"Seed size: {seed_size}")
k = seed_size
max_iter = 1000
budget = 3 * seed_size

Seed size: 250


In [9]:
# Define your seed functions
seed_functions = {
    "Random": seed_random,
    "MIA": lambda G, seed_size: seed_influence_maximization(G, seed_size, 200, 8, 1),
    "Degree": seed_degree,
    "Centrality": seed_centrality,
    "Centrality Mixed": seed_centrality_mixed,
    "Polarized": lambda G, seed_size: seed_polarized(G, seed_size, 0),
    "Polarized Degree": lambda G, seed_size: seed_polarized_degree(G, seed_size, 0),
    "Polarized Centrality": lambda G, seed_size: seed_polarized_centrality(G, seed_size, 0),
    "Polarized Centrality Mixed": lambda G, seed_size: seed_polarized_centrality_mixed(G, seed_size, 0),
}

# Example of how to call the function
all_results_df = evaluate_all_seeds(G, seed_functions, seed_size, k, max_iter, budget, "netin_01_2")

# Display the combined results
print("Combined Results for All Seed Functions")
all_results_df

Running evaluation for seed function: Random
    Running evaluation for graph modification: Original Graph


100%|██████████| 1000/1000 [00:15<00:00, 62.62it/s]


    Running evaluation for graph modification: Random


100%|██████████| 1000/1000 [00:18<00:00, 54.29it/s] 


    Running evaluation for graph modification: Degree


...

100%|██████████| 1000/1000 [00:31<00:00, 31.98it/s]


    Running evaluation for graph modification: Custom V2


100%|██████████| 1000/1000 [00:31<00:00, 31.49it/s]

Combined Results for All Seed Functions


Metric,Seed Function,Graph Modification,Avg Activated Nodes,Activated Nodes Std Dev,"Avg Activated Nodes, Color (0)","Activated Nodes Std Dev, Color (0)","Avg Activated Nodes, Color (1)","Activated Nodes Std Dev, Color (1)",Avg Color Activation Count,Color Activation Count Std Dev,Number of Nodes,Number of Edges
0,Centrality,Original Graph,584.007,166.201,231.833,67.640,352.174,99.816,282.678,131.642,10000.0,99990.0
1,Centrality,PrefAtt,9119.000,0.000,6325.000,0.000,2794.000,0.000,3692.683,436.610,10000.0,100740.0
2,Centrality,Jaccard,9119.000,0.000,6325.000,0.000,2794.000,0.000,3353.608,339.864,10000.0,100740.0
3,Centrality,Degree,9119.000,0.000,6325.000,0.000,2794.000,0.000,8552.962,1807.966,10000.0,100740.0
4,Centrality,TopK,9119.000,0.000,6325.000,0.000,2794.000,0.000,9465.092,2282.771,10000.0,100740.0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Random,TopK,9140.000,0.000,6341.000,0.000,2799.000,0.000,4303.281,376.067,10000.0,100740.0
77,Random,KKT,1906.328,291.110,860.026,147.898,1046.302,146.427,1837.716,413.667,10000.0,100740.0
78,Random,Random,1154.898,172.902,579.672,77.619,575.226,97.682,844.985,140.919,10000.0,100740.0
79,Random,Custom,1302.479,180.820,584.876,83.258,717.603,100.390,1098.715,153.414,10000.0,100740.0


In [10]:
#all_results_df.to_csv(f"test.csv", index=False)
all_results_df.to_csv(f"results/results_netin_{N}_nodes_{h}_hvalue_{d}_density_{plo}_plo_2.csv", index=False)